# Assignment 1

## Data cleaning and replacing string datatype with integer (which can be fed as a feature to our ANN) in columns Gender and Geography

In [188]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [189]:
from google.colab import files

uploaded = files.upload()

Saving dataset.csv to dataset (2).csv


In [202]:
customer_dataset_df = pd.read_csv('dataset.csv', delimiter=',')

In [203]:
#customer_dataset_df.replace()

#### Handling string datatype in column Geography

Spain: 0, France: 1, Germany: 2

#### Handling the Gender column

Male: 0
Female: 1


In [316]:
def handle_string_type_columns(customer_dataset_df):
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'Spain', 'Geography'] = 0
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'France', 'Geography'] = 1
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'Germany', 'Geography'] = 2

  customer_dataset_df.loc[customer_dataset_df['Gender']== 'Male', 'Gender'] = 0
  customer_dataset_df.loc[customer_dataset_df['Gender']== 'Female', 'Gender'] = 1

  return customer_dataset_df

In [317]:
customer_dataset_df= handle_string_type_columns(customer_dataset_df)

In [318]:
customer_dataset_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.308,0.0,0.0,0.216216,0.0,0.219136,0.000000,1.0,1.0,0.684433,0.0
1,0.618,0.5,1.0,0.202703,0.7,0.358471,0.000000,1.0,0.0,0.682583,0.0
2,0.246,1.0,1.0,0.189189,0.5,0.584310,0.333333,1.0,1.0,0.364462,0.0
3,0.574,0.0,1.0,0.297297,0.6,0.000000,0.333333,1.0,1.0,0.908046,0.0
4,0.614,0.0,0.0,0.770270,0.7,0.503288,0.000000,0.0,1.0,0.458141,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8995,0.808,0.5,1.0,0.121622,0.7,0.000000,0.333333,1.0,0.0,0.720574,0.0
8996,0.718,1.0,0.0,0.162162,0.9,0.460264,0.333333,1.0,1.0,0.673543,0.0
8997,0.742,0.5,0.0,0.486486,0.5,0.000000,0.333333,1.0,1.0,0.022026,0.0
8998,0.490,1.0,0.0,0.162162,0.9,0.520857,0.333333,1.0,1.0,0.289005,0.0


In [207]:
customer_dataset_df= customer_dataset_df.drop(columns=['CustomerId', 'Surname'])

## Scaling: Applying MinMax Data Scaling to handle columns that have large continuous values like EstimatedSalary

In [208]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [209]:
customer_dataset_df[['CreditScore']]= scaler.fit_transform(customer_dataset_df[['CreditScore']])

In [210]:
customer_dataset_df[customer_dataset_df.columns]= scaler.fit_transform(customer_dataset_df[customer_dataset_df.columns])

In [211]:
customer_dataset_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.308,0.0,0.0,0.216216,0.0,0.219136,0.000000,1.0,1.0,0.684433,0.0
1,0.618,0.5,1.0,0.202703,0.7,0.358471,0.000000,1.0,0.0,0.682583,0.0
2,0.246,1.0,1.0,0.189189,0.5,0.584310,0.333333,1.0,1.0,0.364462,0.0
3,0.574,0.0,1.0,0.297297,0.6,0.000000,0.333333,1.0,1.0,0.908046,0.0
4,0.614,0.0,0.0,0.770270,0.7,0.503288,0.000000,0.0,1.0,0.458141,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8995,0.808,0.5,1.0,0.121622,0.7,0.000000,0.333333,1.0,0.0,0.720574,0.0
8996,0.718,1.0,0.0,0.162162,0.9,0.460264,0.333333,1.0,1.0,0.673543,0.0
8997,0.742,0.5,0.0,0.486486,0.5,0.000000,0.333333,1.0,1.0,0.022026,0.0
8998,0.490,1.0,0.0,0.162162,0.9,0.520857,0.333333,1.0,1.0,0.289005,0.0


## Test-train Split

In [212]:
Y_df= customer_dataset_df["Exited"]


In [213]:
X_df= customer_dataset_df.drop(columns=['Exited'])

In [214]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, test_size=0.2, random_state=7)

Converting df to numpy array (accepted by keras's model.fit(..) )

In [215]:
X_train_numpy= X_train.to_numpy()
y_train_numpy= y_train.to_numpy()
X_test_numpy= X_test.to_numpy()
y_test_numpy= y_test.to_numpy()



Converting all int values to float (enabling numpy array to be converted to tensor when fed into model.fit(..) ) 

In [216]:
X_train_numpy = np.asarray(X_train_numpy).astype('float32')
y_train_numpy = np.asarray(y_train_numpy).astype('float32')

X_test_numpy = np.asarray(X_test_numpy).astype('float32')
y_test_numpy = np.asarray(y_test_numpy).astype('float32')

## Building Keras Sequential Models

In [217]:
import tensorflow as tf

In [218]:
from tensorflow.keras import datasets, layers, models, Sequential, optimizers, regularizers

In [219]:
BATCH_SIZE = 32

In [276]:
def calculate_F1(precision, recall):
  F1_score = 2 * (precision * recall) / (precision + recall)
  return F1_score


## Model A

In [277]:
model_A= Sequential()

In [278]:
model_A.add(tf.keras.Input(shape=(10,)))
model_A.add(layers.Dense(64, activation='relu'))6
model_A.add(layers.Dense(32, activation='relu'))
model_A.add(layers.Dense(32, activation='relu'))
model_A.add(layers.Dense(16, activation='relu'))
model_A.add(layers.Dense(8, activation='relu'))
model_A.add(layers.Dense(1, activation='sigmoid'))

In [279]:
model_A.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 64)                704       
_________________________________________________________________
dense_92 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_93 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_94 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_95 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_96 (Dense)             (None, 1)                 9         
Total params: 4,513
Trainable params: 4,513
Non-trainable params: 0
___________________________________________________

In [280]:
model_A.compile(optimizer= optimizers.Adam(lr=0.01),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5), tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.Precision(thresholds=0.5), tf.keras.metrics.Recall(thresholds=0.5)] )

In [281]:
model_A.fit(x= X_train_numpy, y=y_train_numpy, batch_size=BATCH_SIZE, epochs=80, verbose=2, validation_data= (X_test_numpy,y_test_numpy))

Epoch 1/80
225/225 - 2s - loss: 0.4712 - binary_accuracy: 0.7954 - binary_crossentropy: 0.4712 - precision_4: 0.7143 - recall_4: 0.0034 - val_loss: 0.4583 - val_binary_accuracy: 0.8028 - val_binary_crossentropy: 0.4583 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/80
225/225 - 0s - loss: 0.4204 - binary_accuracy: 0.8125 - binary_crossentropy: 0.4204 - precision_4: 0.6189 - recall_4: 0.2222 - val_loss: 0.3992 - val_binary_accuracy: 0.8267 - val_binary_crossentropy: 0.3992 - val_precision_4: 0.9057 - val_recall_4: 0.1352
Epoch 3/80
225/225 - 0s - loss: 0.3827 - binary_accuracy: 0.8358 - binary_crossentropy: 0.3827 - precision_4: 0.7019 - recall_4: 0.3462 - val_loss: 0.3590 - val_binary_accuracy: 0.8561 - val_binary_crossentropy: 0.3590 - val_precision_4: 0.7963 - val_recall_4: 0.3634
Epoch 4/80
225/225 - 0s - loss: 0.3675 - binary_accuracy: 0.8479 - binary_crossentropy: 0.3675 - precision_4: 0.7606 - recall_4: 0.3767 - val_loss: 0.3637 - val_binary_accuracy: 0.8544 - v

In [309]:
Train_F1= calculate_F1(0.7840,0.5115)

print("Train F1: "+ str(Train_F1) )

Train F1: 0.6190906985719798


In [306]:
loss, acc, binary_crossentropy, precision, recall= model_A.evaluate(X_test_numpy, y_test_numpy)
print("Test Accuracy: "+ str(acc))
print("Test Precision: " + str(precision))
print('Test Recall: '+ str(recall))

F1= calculate_F1(precision, recall)
print("Test F1: "+ str(F1))

57/57 [==============================] - 0s 1ms/step - loss: 0.3821 - binary_accuracy: 0.8567 - binary_crossentropy: 0.3821 - precision_4: 0.7299 - recall_4: 0.4338
Test Accuracy: 0.8566666841506958
Test Precision: 0.7298578023910522
Test Recall: 0.4338028132915497
F1: 0.5441696035992433


In [254]:
#-------------------------------------------------
from sklearn.metrics import classification_report
b= np.array( [0.7] )
print( np.round(b) )

y_predicted= model_A.predict(X_test_numpy)
y_predicted

rounded_y_predicted= np.round(y_predicted)
#print( classification_report(y_test_numpy, rounded_y_predicted) ) 


Num Epochs: 80
Test Accuracy: 85.6

## Model B

In [245]:
model_B= Sequential()
model_B.add(tf.keras.Input(shape=(10,)))
model_B.add(layers.Dense(128, activation='relu'))
model_B.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model_B.add(layers.Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model_B.add(layers.Dense(8, activation='sigmoid'))
model_B.add(layers.Dense(1, activation='sigmoid'))


In [296]:
model_B.compile(optimizer= optimizers.Adam(lr=0.01),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5), tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.Precision(thresholds=0.5), tf.keras.metrics.Recall(thresholds=0.5)] )

In [247]:
model_B.fit(x= X_train_numpy, y=y_train_numpy, batch_size=BATCH_SIZE, epochs=120, verbose=2, validation_data= (X_test_numpy,y_test_numpy))

Epoch 1/120
225/225 - 2s - loss: 0.5720 - binary_accuracy: 0.7846 - binary_crossentropy: 0.5072 - precision_2: 0.2378 - recall_2: 0.0230 - val_loss: 0.4824 - val_binary_accuracy: 0.8028 - val_binary_crossentropy: 0.4690 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/120
225/225 - 0s - loss: 0.4682 - binary_accuracy: 0.8090 - binary_crossentropy: 0.4553 - precision_2: 0.6312 - recall_2: 0.1646 - val_loss: 0.4529 - val_binary_accuracy: 0.8122 - val_binary_crossentropy: 0.4425 - val_precision_2: 0.7429 - val_recall_2: 0.0732
Epoch 3/120
225/225 - 0s - loss: 0.4472 - binary_accuracy: 0.8188 - binary_crossentropy: 0.4353 - precision_2: 0.6482 - recall_2: 0.2534 - val_loss: 0.4154 - val_binary_accuracy: 0.8389 - val_binary_crossentropy: 0.3998 - val_precision_2: 0.7044 - val_recall_2: 0.3155
Epoch 4/120
225/225 - 0s - loss: 0.4095 - binary_accuracy: 0.8403 - binary_crossentropy: 0.3923 - precision_2: 0.7063 - recall_2: 0.3780 - val_loss: 0.3821 - val_binary_accuracy: 0.8506

In [312]:
Train_F1= calculate_F1(0.7604,0.4837)

print("Train F1: "+ str(Train_F1) )

Train F1: 0.5912796077485734


In [310]:
loss, acc, binary_crossentropy, precision, recall= model_B.evaluate(X_test_numpy, y_test_numpy)
print("Test Accuracy: "+ str(acc))
print("Test Precision: " + str(precision))
print('Test Recall: '+ str(recall))
F1= calculate_F1(precision, recall)
print("F1: "+ str(F1))

57/57 [==============================] - 0s 1ms/step - loss: 0.3520 - binary_accuracy: 0.8600 - binary_crossentropy: 0.3448 - precision_5: 0.7977 - recall_5: 0.3887
Test Accuracy: 0.8600000143051147
Test Precision: 0.7976878881454468
Test Recall: 0.3887324035167694
F1: 0.5227272867704456


Num Epochs: 120

Best Test Accuracy: 0.86

## Model C

In [300]:
model_C= Sequential()
model_C.add(tf.keras.Input(shape=(10,)))
model_C.add(layers.Dense(512, activation='relu'))
model_C.add(layers.Dense(256, activation='relu'))
model_C.add(layers.Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model_C.add(layers.Dense(1, activation='sigmoid'))

In [301]:
model_C.compile(optimizer= optimizers.Adam(lr=0.01),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5), tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.Precision(thresholds=0.5), tf.keras.metrics.Recall(thresholds=0.5)] )

In [302]:
model_C.fit(x= X_train_numpy, y=y_train_numpy, batch_size=BATCH_SIZE, epochs=300, verbose=2, validation_data= (X_test_numpy,y_test_numpy))

Epoch 1/300
225/225 - 2s - loss: 0.4886 - binary_accuracy: 0.7918 - binary_crossentropy: 0.4679 - precision_6: 0.0741 - recall_6: 0.0014 - val_loss: 0.4403 - val_binary_accuracy: 0.8122 - val_binary_crossentropy: 0.4360 - val_precision_6: 0.7576 - val_recall_6: 0.0704
Epoch 2/300
225/225 - 1s - loss: 0.4260 - binary_accuracy: 0.8228 - binary_crossentropy: 0.4207 - precision_6: 0.6748 - recall_6: 0.2615 - val_loss: 0.3878 - val_binary_accuracy: 0.8461 - val_binary_crossentropy: 0.3812 - val_precision_6: 0.8750 - val_recall_6: 0.2563
Epoch 3/300
225/225 - 1s - loss: 0.3864 - binary_accuracy: 0.8431 - binary_crossentropy: 0.3793 - precision_6: 0.7247 - recall_6: 0.3780 - val_loss: 0.4009 - val_binary_accuracy: 0.8578 - val_binary_crossentropy: 0.3937 - val_precision_6: 0.7647 - val_recall_6: 0.4028
Epoch 4/300
225/225 - 1s - loss: 0.3731 - binary_accuracy: 0.8449 - binary_crossentropy: 0.3671 - precision_6: 0.7186 - recall_6: 0.3997 - val_loss: 0.3655 - val_binary_accuracy: 0.8561 - val_b

In [313]:
Train_F1= calculate_F1(0.8694, 0.5955)

print("Train F1: "+ str(Train_F1) )

Train F1: 0.7068437436002457


In [314]:
loss, acc, binary_crossentropy, precision, recall= model_C.evaluate(X_test_numpy, y_test_numpy)
print("Test Accuracy: "+ str(acc))
print("Test Precision: " + str(precision))
print('Test Recall: '+ str(recall))
F1= calculate_F1(precision, recall)
print("Test F1: "+ str(F1))

57/57 [==============================] - 0s 2ms/step - loss: 0.5698 - binary_accuracy: 0.8378 - binary_crossentropy: 0.5587 - precision_6: 0.6145 - recall_6: 0.4761
Test Accuracy: 0.8377777934074402
Test Precision: 0.614545464515686
Test Recall: 0.4760563373565674
Test F1: 0.5365079398808775


Num Epochs: 300

 Test Accuracy: 83.77
 